## Author: Dwane Richards, John Rielley, Mustafa Radheyyan, Jose Heras, Wuhbet Tewelde
## Date: 01/23/2023
## Assignment: Cognixia JUMPro Python Project 2
## Dataset Synopsis: https://www.kaggle.com/datasets/mattop/daily-solar-power-generation
## Dataset Download (warning: link to download): https://www.kaggle.com/datasets/mattop/daily-solar-power-generation/download?datasetVersionNumber=1

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from IPython.display import display, HTML

In [ ]:
solar_output = pd.read_csv("solar.csv", header=0)

Table from the unmodified csv

In [ ]:
solar_output.head()

Modifying the table, removing unneeded columns and removing NaN values.

In [ ]:
solar_output = solar_output.drop(labels=["latitude", "longitude", "department", "address", "site"], axis=1)
solar_output = solar_output.dropna(subset=["watt_hours", "date", "watt_hours_expected"], axis='rows').reset_index(drop=True)
solar_output.head()

Our dataset still appears to have a large number of NaN values in the "watt_min" column. Therefore, we have opted to do some investigation into the column.

In [ ]:
solar_output["watt_min"].max()

In [ ]:
solar_output = solar_output.drop(labels=["watt_min"], axis=1).reset_index(drop=True)
solar_output.head()

Let's examine the DataFrame's data types!

The date column's data type is an object! What could make that better? Datetime possibly...?

In [ ]:
solar_output.info()

Datetime it is! Much cleaner!

In [ ]:
solar_output["date"] = pd.to_datetime(solar_output["date"])
solar_output.head()

Calculating the difference between expected watt hours and actual watt hour output

Consecutive dates with large discrepancies in watt hour outputs such as between 2021-05-08 and 2021-05-09 may be explained by a broken device or sensor

In [ ]:
solar_output["watt_hours_expected_difference"] = solar_output["watt_hours"] - solar_output["watt_hours_expected"]
solar_output_display = solar_output[(solar_output['date'] < dt.datetime(2018, 1, 25)) | (solar_output['date'] > dt.datetime(2021, 5, 6))].reset_index(drop=True)

dateSlice = solar_output_display[(solar_output_display['date'].eq(dt.datetime(2021, 5, 8))) | (solar_output_display['date'].eq(dt.datetime(2021, 5, 9)))]
last_row = pd.IndexSlice[dateSlice.index,:]

def df_style(val): return "font-weight: bold"
solar_output_display = solar_output_display.style.set_properties(**{'background-color': 'green'}, subset=last_row)
solar_output_display = solar_output_display.applymap(df_style, subset=last_row)
solar_output_display\
    .hide(axis="index").format(
    {
        'date': lambda t: t.strftime("%Y-%m-%d"),
        'watt_max':'{:.0f}',
        'watt_avg':'{:.0f}',
        'watt_hours':'{:.0f}',
        'watt_hours_expected':'{:.0f}',
        'watt_hours_expected_difference':'{:.0f}'
     })

A boolean representation of when watt hour output exceeded expected watt hours

In [ ]:
solar_output["watt_hours_exceeded_expected"] = np.where(solar_output["watt_hours"] >= solar_output["watt_hours_expected"], True, False)
solar_output

In [ ]:
average_solar_output_over_expected = round(solar_output["watt_hours_expected_difference"].mean(), 2)

analysis_string = f"""<h2 style='font-size:18'>Through our dataset, the data has produced, on average, more power than expected.

<br><br>It is currently producing on average: <span><span style='color:red'>{average_solar_output_over_expected}</span> watt hours over the expected watt hours.

<br><br>In the future, the station is predicted, on average, to overproduce.</span>"""

# Native Python print version, smaller text size =(
# analysis_string = f"""Through our dataset, the data has produced on average more power than expected.
# It is currently producing on average: {average_solar_output_over_expected} watt hours over the expected watt hours.
# In the future, the station is predicted to overproduce on average."""

# print(analysis_string)

display(HTML(analysis_string))

Converting the date into months, and finding the average difference between watt hour output and expected watt hours.

This monthly breakdown of the data shows that the solar panels produced well above expected watt hours in September, and in every month
except for May, August, December and November 

In [ ]:
date_column_month = pd.DataFrame({'month' : solar_output["date"].dt.month_name()})
solar_output_month = date_column_month.join(solar_output)
solar_output_month[['month', 'watt_hours_expected_difference']].groupby("month").mean().round(decimals=2).sort_values(by="watt_hours_expected_difference", ascending=False)

This monthly breakdown of the data shows that the solar panels had higher average watt hours in summer and spring months, rather than in winter months.

In [ ]:
date_column_month = pd.DataFrame({'month' : solar_output["date"].dt.month_name()})
solar_output_month = date_column_month.join(solar_output)
solar_output_month[['month', 'watt_avg']].groupby("month").mean().round(decimals=2).sort_values(by="watt_avg", ascending=False)

Highest Expected Watt Output

In [ ]:
solar_output[['date', 'watt_hours_expected']].sort_values(['watt_hours_expected'], ascending=False).head()\
    .style.hide(axis="index").format(
    {
        'date': lambda t: t.strftime("%Y-%m-%d"),
        'watt_hours_expected':'{:.0f}'
     })

Lowest Expected Watt Output

In [ ]:
solar_output[['date', 'watt_hours_expected']].sort_values(['watt_hours_expected'], ascending=False).dropna().tail()\
    .style.hide(axis="index").format(
    {
        'date': lambda t: t.strftime("%Y-%m-%d"),
        'watt_hours_expected':'{:.0f}'
     })

Highest average watt output

In [ ]:
solar_output[['date', 'watt_avg']].sort_values(['watt_avg'], ascending=False).head()\
    .style.hide(axis="index").format(
    {
        'date': lambda t: t.strftime("%Y-%m-%d"),
        'watt_avg':'{:.0f}'
     })

Lowest average watt output

In [ ]:
lowest_avg_watts = solar_output[['date', 'watt_avg']].sort_values(['watt_avg'], ascending=False).dropna()
lowest_avg_watts[lowest_avg_watts["watt_avg"] > 0].tail()\
    .style.hide(axis="index").format(
    {
        'date': lambda t: t.strftime("%Y-%m-%d"),
        'watt_avg':'{:.0f}'
     })